# signals

In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit

In [3]:
# Disable caching for performance testing
vbt.settings.caching['enabled'] = False

## accessors

In [4]:
index = pd.Index([
    datetime(2018, 1, 1),
    datetime(2018, 1, 2),
    datetime(2018, 1, 3),
    datetime(2018, 1, 4),
    datetime(2018, 1, 5)
])
columns = ['a', 'b', 'c']
big_index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]

In [5]:
entries = pd.DataFrame({
    'a': [True, False, False, False, False],
    'b': [True, False, True, False, True],
    'c': [True, True, True, False, False],
}, index=index)
print(entries.shape)

big_entries = pd.DataFrame(np.full((1000, 1000), False), index=big_index)
big_entries.iloc[::10] = True
print(big_entries.shape)

In [6]:
ts = pd.Series([1., 2., 3., 2., 1.], index=index, name=columns[0])
print(ts.shape)

big_ts = pd.Series(np.random.uniform(10, 13, size=(1000,)), index=big_index)
print(big_ts.shape)

In [7]:
price = pd.DataFrame({
    'open': [10, 11, 12, 11, 10],
    'high': [11, 12, 13, 12, 11],
    'low': [9, 10, 11, 10, 9],
    'close': [10, 11, 12, 11, 10]
})

In [8]:
a = np.random.randint(-1, 2, size=(5, 3))
print(a.shape)

big_a = np.random.randint(-1, 2, size=(1000, 1000))
print(big_a.shape)

In [9]:
print(entries.vbt.signals.shuffle(seed=42))

%timeit big_entries.vbt.signals.shuffle(seed=42)

In [10]:
print(entries.vbt.signals.fshift(2))

%timeit big_entries.vbt.signals.fshift(2)

In [11]:
print(pd.Series.vbt.signals.empty(5, index=index))
print(pd.DataFrame.vbt.signals.empty((5, 3), index=index, columns=columns))

%timeit pd.DataFrame.vbt.signals.empty((1000, 1000))

In [12]:
@njit
def choice_func_nb(from_i, to_i, col):
    return np.random.choice(np.arange(from_i, to_i), size=1, replace=False)

print(pd.Series.vbt.signals.generate(5, choice_func_nb, index=index))
print(pd.DataFrame.vbt.signals.generate((5, 3), choice_func_nb, index=index, columns=columns))

%timeit pd.DataFrame.vbt.signals.generate((1000, 1000), choice_func_nb)

In [14]:
@njit
def entry_func_nb(from_i, to_i, col, temp_int):
    temp_int[0] = from_i
    return temp_int[:1]

@njit
def exit_func_nb(from_i, to_i, col, temp_int):
    temp_int[0] = from_i
    return temp_int[:1]

temp_int = np.empty((1000,), dtype=np.int_)
en, ex = pd.Series.vbt.signals.generate_both(
    a.shape[0], entry_func_nb, (temp_int,), exit_func_nb, (temp_int,), 
    index=index)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_both(
    a.shape, entry_func_nb, (temp_int,), exit_func_nb, (temp_int,), 
    index=index, columns=columns)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_both(
    a.shape, entry_func_nb, (temp_int,), exit_func_nb, (temp_int,), 
    index=index, columns=columns, entry_wait=1, exit_wait=0)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_both(
    a.shape, entry_func_nb, (temp_int,), exit_func_nb, (temp_int,), 
    index=index, columns=columns, entry_wait=0, exit_wait=1)
print(en)
print(ex)

%timeit pd.DataFrame.vbt.signals.generate_both(\
    big_a.shape, entry_func_nb, (temp_int,), exit_func_nb, (temp_int,))

In [15]:
@njit
def choice_func_nb(from_i, to_i, col, temp_int):
    temp_int[0] = from_i
    return temp_int[:1]

print(entries.vbt.signals.generate_exits(choice_func_nb, temp_int))
print(entries.vbt.signals.generate_exits(choice_func_nb, temp_int, wait=0))

%timeit big_entries.vbt.signals.generate_exits(choice_func_nb, temp_int)

In [16]:
print(pd.Series.vbt.signals.generate_random(5, n=2, seed=42, index=index))
print(pd.DataFrame.vbt.signals.generate_random((5, 3), n=2, seed=42, index=index, columns=columns))
print(pd.DataFrame.vbt.signals.generate_random((5, 3), n=[0, 1, 2], seed=42, index=index, columns=columns))

%timeit pd.DataFrame.vbt.signals.generate_random((1000, 1000), n=100)

In [17]:
print(pd.Series.vbt.signals.generate_random(5, prob=0.5, seed=42, index=index))
print(pd.DataFrame.vbt.signals.generate_random((5, 3), prob=0.5, seed=42, index=index, columns=columns))
print(pd.DataFrame.vbt.signals.generate_random((5, 3), prob=[0., 0.5, 1], seed=42, index=index, columns=columns))

%timeit pd.DataFrame.vbt.signals.generate_random((1000, 1000), prob=0.5)

In [18]:
print(entries['a'].vbt.signals.generate_random_exits(seed=42))
print(entries.vbt.signals.generate_random_exits(seed=42))
print(entries.vbt.signals.generate_random_exits(seed=42, wait=0))

%timeit big_entries.vbt.signals.generate_random_exits(seed=42)

In [19]:
print(entries['a'].vbt.signals.generate_random_exits(prob=1., seed=42))
print(entries.vbt.signals.generate_random_exits(prob=1., seed=42))
print(entries.vbt.signals.generate_random_exits(prob=[0., 0.5, 1], seed=42))
print(entries.vbt.signals.generate_random_exits(prob=1., seed=42, wait=0))

%timeit big_entries.vbt.signals.generate_random_exits(prob=1., seed=42)

In [20]:
en, ex = pd.Series.vbt.signals.generate_random_both(5, n=2, seed=42, index=index)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both((5, 3), n=2, seed=42, index=index, columns=columns)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both((5, 3), n=[0, 1, 2], seed=42, index=index, columns=columns)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both((2, 3), n=2, seed=42, entry_wait=1, exit_wait=0)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both((3, 3), n=2, seed=42, entry_wait=0, exit_wait=1)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both((7, 3), n=2, seed=42, entry_wait=2, exit_wait=2)
print(en)
print(ex)

%timeit pd.DataFrame.vbt.signals.generate_random_both((1000, 1000), n=100)
%timeit pd.DataFrame.vbt.signals.generate_random_both((1000, 1000), n=100, exit_wait=0)

In [21]:
n = 10
a = np.full(n * 2, 0.)
for i in range(10000):
    en, ex = pd.Series.vbt.signals.generate_random_both(1000, n, entry_wait=2, exit_wait=2)
    _a = np.empty((n * 2,), dtype=np.int_)
    _a[0::2] = np.flatnonzero(en)
    _a[1::2] = np.flatnonzero(ex)
    a += _a
print(a)

b = np.full(n * 2, 0.)
for i in range(10000):
    b += np.sort(np.random.choice(1000, size=n * 2, replace=False))
print(b)

In [22]:
en, ex = pd.Series.vbt.signals.generate_random_both(
    5, entry_prob=0.5, exit_prob=1., seed=42, index=index)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both(
    (5, 3), entry_prob=0.5, exit_prob=1., seed=42, index=index, columns=columns)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both(
    (5, 3), entry_prob=[0., 0.5, 1.], exit_prob=[0., 0.5, 1.], seed=42, index=index, columns=columns)
print(en)
print(ex)
en, ex = pd.DataFrame.vbt.signals.generate_random_both(
    (5, 3), entry_prob=1., exit_prob=1., seed=42, index=index, columns=columns, exit_wait=0)
print(en)
print(ex)

%timeit pd.DataFrame.vbt.signals.generate_random_both(\
    (1000, 1000), entry_prob=1., exit_prob=1.)

In [26]:
e = pd.Series([True, False, False, False, False, False])
t = pd.Series([2, 3, 4, 3, 2, 1]).astype(np.float64)

print(e.vbt.signals.generate_stop_exits(t, -0.1))
print(e.vbt.signals.generate_stop_exits(t, -0.1, trailing=True))
print(e.vbt.signals.generate_stop_exits(t, -0.1, trailing=True, pick_first=False))
print(e.vbt.signals.generate_stop_exits(t.vbt.tile(3), [np.nan, -0.5, -1.], trailing=True, pick_first=False))
print(e.vbt.signals.generate_stop_exits(t, -0.1, trailing=True, exit_wait=3))

print(e.vbt.signals.generate_stop_exits(4 - t, 0.1))
print(e.vbt.signals.generate_stop_exits(4 - t, 0.1, trailing=True))
print(e.vbt.signals.generate_stop_exits(4 - t, 0.1, trailing=True, pick_first=False))
print(e.vbt.signals.generate_stop_exits((4 - t).vbt.tile(3), [np.nan, 0.5, 1.], trailing=True, pick_first=False))
print(e.vbt.signals.generate_stop_exits(4 - t, 0.1, trailing=True, exit_wait=3))

%timeit big_entries.vbt.signals.generate_stop_exits(big_ts, -0.1)
%timeit big_entries.vbt.signals.generate_stop_exits(big_ts, -0.1, trailing=True)
%timeit big_entries.vbt.signals.generate_stop_exits(big_ts, -0.1, trailing=True, pick_first=False)

In [28]:
e = pd.Series([True, True, True, True, True, True])
t = pd.Series([2, 3, 4, 3, 2, 1]).astype(np.float64)

en, ex = e.vbt.signals.generate_stop_exits(t, -0.1, trailing=True, chain=True)
print(en)
print(ex)
en, ex = e.vbt.signals.generate_stop_exits(t, -0.1, trailing=True, entry_wait=2, chain=True)
print(en)
print(ex)
en, ex = e.vbt.signals.generate_stop_exits(t, -0.1, trailing=True, exit_wait=2, chain=True)
print(en)
print(ex)

%timeit big_entries.vbt.signals.generate_stop_exits(big_ts, -0.1, chain=True)
%timeit big_entries.vbt.signals.generate_stop_exits(big_ts, -0.1, trailing=True, chain=True)
%timeit big_entries.vbt.signals.generate_stop_exits(big_ts, -0.1, trailing=True, pick_first=False, chain=True)

In [32]:
def test_ohlc_stop_exits(**kwargs):
    out_dict = {}
    result = entries.vbt.signals.generate_ohlc_stop_exits(
        price['open'], price['high'], price['low'], price['close'],
        out_dict=out_dict, **kwargs
    )
    if isinstance(result, tuple):
        _, ex = result
    else:
        ex = result
    out_dict['stop_price'][~ex] = np.nan
    out_dict['stop_type'][~ex] = -1
    return result, out_dict['stop_price'], out_dict['stop_type']

ex, stop_price, stop_type = test_ohlc_stop_exits()
print(ex)
print(stop_price)
print(stop_type)

ex, stop_price, stop_type = test_ohlc_stop_exits(sl_stop=0.1)
print(ex)
print(stop_price)
print(stop_type)

ex, stop_price, stop_type = test_ohlc_stop_exits(sl_stop=0.1, sl_trail=True)
print(ex)
print(stop_price)
print(stop_type)

ex, stop_price, stop_type = test_ohlc_stop_exits(tp_stop=0.1)
print(ex)
print(stop_price)
print(stop_type)

ex, stop_price, stop_type = test_ohlc_stop_exits(sl_stop=0.1, sl_trail=True, tp_stop=0.1)
print(ex)
print(stop_price)
print(stop_type)

ex, stop_price, stop_type = test_ohlc_stop_exits(
    sl_stop=[np.nan, 0.5, 1.], sl_trail=True, tp_stop=[np.nan, 0.5, 1.])
print(ex)
print(stop_price)
print(stop_type)

ex, stop_price, stop_type = test_ohlc_stop_exits(sl_stop=0.1, sl_trail=True, tp_stop=0.1, exit_wait=0)
print(ex)
print(stop_price)
print(stop_type)

%timeit big_entries.vbt.signals.generate_ohlc_stop_exits(\
    big_ts, big_ts + 1, big_ts - 1, big_ts, sl_stop=0.1, sl_trail=True, tp_stop=0.1)

In [33]:
(en, ex), stop_price, stop_type = test_ohlc_stop_exits(sl_stop=0.1, sl_trail=True, tp_stop=0.1, chain=True)
print(en)
print(ex)
print(stop_price)
print(stop_type)

%timeit big_entries.vbt.signals.generate_ohlc_stop_exits(\
    big_ts, big_ts + 1, big_ts - 1, big_ts, sl_stop=0.1, sl_trail=True, tp_stop=0.1, chain=True)

In [35]:
distance_map_nb = njit(lambda prev_i, next_i, col: next_i - prev_i)
avg_reduce_nb = njit(lambda col, a: np.nanmean(a))

print(entries.vbt.signals.map_reduce_between(
    range_map_func_nb=distance_map_nb, reduce_func_nb=avg_reduce_nb))

%timeit big_entries.vbt.signals.map_reduce_between(\
    range_map_func_nb=distance_map_nb, reduce_func_nb=avg_reduce_nb)

In [36]:
print(entries.vbt.signals.map_reduce_between(
    other=entries.vbt.signals.fshift(1), 
    range_map_func_nb=distance_map_nb, 
    reduce_func_nb=avg_reduce_nb))

%timeit big_entries.vbt.signals.map_reduce_between(\
    other=big_entries.vbt.signals.fshift(1),\
    range_map_func_nb=distance_map_nb,\
    reduce_func_nb=avg_reduce_nb)

In [37]:
print(entries.vbt.signals.map_reduce_partitions(
    range_map_func_nb=distance_map_nb, reduce_func_nb=avg_reduce_nb))

%timeit big_entries.vbt.signals.map_reduce_partitions(\
    range_map_func_nb=distance_map_nb, reduce_func_nb=avg_reduce_nb)

In [41]:
print(entries['a'].vbt.signals.total())
print(entries.vbt.signals.total())

%timeit big_entries.vbt.signals.total()

In [42]:
print(entries['a'].vbt.signals.avg_distance())
print(entries.vbt.signals.avg_distance())

%timeit big_entries.vbt.signals.avg_distance()

In [43]:
print(entries['a'].vbt.signals.avg_distance(to=entries['a'].vbt.signals.fshift(1)))
print(entries.vbt.signals.avg_distance(to=entries.vbt.signals.fshift(1)))

%timeit big_entries.vbt.signals.avg_distance(to=big_entries.vbt.signals.fshift(1))

In [45]:
print(entries['a'].vbt.signals.pos_rank())
print(entries.vbt.signals.pos_rank())

print(entries['a'].vbt.signals.pos_rank(after_false=True))
print(entries.vbt.signals.pos_rank(after_false=True))

print(entries['a'].vbt.signals.pos_rank(allow_gaps=True))
print(entries.vbt.signals.pos_rank(allow_gaps=True))

print(entries['a'].vbt.signals.pos_rank(allow_gaps=True, reset_by=~entries['a']))
print(entries.vbt.signals.pos_rank(allow_gaps=True, reset_by=~entries))

%timeit big_entries.vbt.signals.pos_rank()

In [47]:
print(entries['a'].vbt.signals.partition_pos_rank())
print(entries.vbt.signals.partition_pos_rank())

print(entries['a'].vbt.signals.partition_pos_rank(after_false=True))
print(entries.vbt.signals.partition_pos_rank(after_false=True))

print(entries['a'].vbt.signals.partition_pos_rank(reset_by=~entries['a']))
print(entries.vbt.signals.partition_pos_rank(reset_by=~entries))

%timeit big_entries.vbt.signals.partition_pos_rank()

In [48]:
print(entries.vbt.signals.first())

%timeit big_entries.vbt.signals.first()

In [49]:
print(entries & entries)
%timeit big_entries & big_entries

print(entries.vbt.signals.AND(entries))
%timeit big_entries.vbt.signals.AND(big_entries) # a bit slower but does smart broadcasting

In [50]:
print(entries.vbt.signals.OR([ts > 1, ts > 2, ts > 3])) # you can pass multiple arguments
print(entries.vbt.signals.OR([ts > 1, ts > 2, ts > 3], concat=True, keys=['>1', '>2', '>3']))

In [56]:
print(entries['a'].vbt.signals.stats())
%timeit big_entries[0].vbt.signals.stats(silence_warnings=True)

print(entries.vbt.signals.stats(column='a'))
%timeit big_entries.vbt.signals.stats(column=0, silence_warnings=True)

print(entries.vbt.signals.stats())
%timeit big_entries.vbt.signals.stats(silence_warnings=True)
                                      
print(entries.vbt.signals.stats(settings=dict(other=~entries)))
%timeit big_entries.vbt.signals.stats(settings=dict(other=~big_entries), silence_warnings=True)

In [57]:
entries.vbt.signals.plot().show_svg()

In [58]:
entries['a'].vbt.signals.plot().show_svg()

In [59]:
entries['a'].vbt.signals.plot_as_markers(ts).show_svg()

In [60]:
entries['a'].vbt.signals.plot_as_entry_markers(ts).show_svg()

In [61]:
entries['a'].vbt.signals.plot_as_exit_markers(ts).show_svg()

## factory

In [63]:
@njit
def choice_nb(from_i, to_i, col, ts, in_out, n, arg, temp_idx_arr, kw):
    in_out[from_i, col] = ts[from_i, col] * n + arg + kw
    temp_idx_arr[0] = from_i
    return temp_idx_arr[:1]

MySignals = vbt.SignalFactory(
    input_names=['ts1', 'ts2'],
    in_output_names=['in_out1', 'in_out2'],
    param_names=['n1', 'n2']
).from_choice_func(
    entry_choice_func=choice_nb,
    entry_settings=dict(
        pass_inputs=['ts1'],
        pass_in_outputs=['in_out1'],
        pass_params=['n1'],
        pass_kwargs=['temp_idx_arr1', ('kw1', 1000)]
    ),
    exit_choice_func=choice_nb,
    exit_settings=dict(
        pass_inputs=['ts2'],
        pass_in_outputs=['in_out2'],
        pass_params=['n2'],
        pass_kwargs=['temp_idx_arr2', ('kw2', 1000)]
    ),
    in_output_settings=dict(
        in_out1=dict(
            dtype=np.float_
        ),
        in_out2=dict(
            dtype=np.float_
        )
    ),
    in_out1=np.nan,
    in_out2=np.nan,
    var_args=True,
    require_input_shape=False
)
my_sig = MySignals.run(np.arange(5), np.arange(5), [0, 1], [1, 0], entry_args=(100,), exit_args=(100,))
print(my_sig.entries)
print(my_sig.exits)
print(my_sig.in_out1)
print(my_sig.in_out2)
my_sig[(0, 1)].plot().show_svg()

In [64]:
my_sig = MySignals.run(
    np.arange(7), np.arange(7), [0, 1], [1, 0], 
    entry_args=(100,), exit_args=(100,), 
    entry_kwargs=dict(wait=2), exit_kwargs=dict(wait=2)
)
print(my_sig.entries)
print(my_sig.exits)
print(my_sig.in_out1)
print(my_sig.in_out2)

In [65]:
MySignals = vbt.SignalFactory(
    input_names=['ts2'],
    in_output_names=['in_out2'],
    param_names=['n2'],
    mode='exits'
).from_choice_func(
    exit_choice_func=choice_nb,
    exit_settings=dict(
        pass_inputs=['ts2'],
        pass_in_outputs=['in_out2'],
        pass_params=['n2'],
        pass_kwargs=['temp_idx_arr2', ('kw2', 1000)]
    ),
    in_output_settings=dict(
        in_out2=dict(
            dtype=np.float_
        )
    ),
    in_out2=np.nan,
    var_args=True
)
e = np.array([True, False, True, False, True])
my_sig = MySignals.run(e, np.arange(5), [1, 0], 100)
print(my_sig.entries)
print(my_sig.exits)
print(my_sig.in_out2)
my_sig[0].plot().show_svg()

In [66]:
e = np.array([True, False, False, True, False, False])
my_sig = MySignals.run(e, np.arange(6), [1, 0], 100, wait=2)
print(my_sig.entries)
print(my_sig.exits)
print(my_sig.in_out2)

In [67]:
MySignals = vbt.SignalFactory(
    input_names=['ts2'],
    in_output_names=['in_out2'],
    param_names=['n2'],
    mode='chain'
).from_choice_func(
    exit_choice_func=choice_nb,
    exit_settings=dict(
        pass_inputs=['ts2'],
        pass_in_outputs=['in_out2'],
        pass_params=['n2'],
        pass_kwargs=['temp_idx_arr2', ('kw2', 1000)]
    ),
    in_output_settings=dict(
        in_out2=dict(
            dtype=np.float_
        )
    ),
    in_out2=np.nan, 
    var_args=True
)
e = np.array([True, True, True, True, True])
my_sig = MySignals.run(e, np.arange(5), [1, 0], 100)
print(my_sig.entries)
print(my_sig.new_entries)
print(my_sig.exits)
print(my_sig.in_out2)
my_sig[0].plot().show_svg()

In [68]:
e = np.array([True, True, True, True, True, True])
my_sig = MySignals.run(e, np.arange(6), [1, 0], 100, wait=2)
print(my_sig.entries)
print(my_sig.new_entries)
print(my_sig.exits)
print(my_sig.in_out2)

## basic

### RANDNX

In [69]:
randnx = vbt.RANDNX.run(n=1, input_shape=(6,), seed=42)

print(randnx.entries)
print(randnx.exits)

In [70]:
randnx = vbt.RANDNX.run(n=[1, 2, 3], input_shape=(6,), seed=42)

print(randnx.entries)
print(randnx.exits)

In [71]:
randnx = vbt.RANDNX.run(n=[np.array([1, 2]), np.array([3, 4])], input_shape=(8, 2), seed=42)

print(randnx.entries)
print(randnx.exits)

In [72]:
%timeit vbt.RANDNX.run(n=100, input_shape=(1000, 1000), seed=42)
%timeit vbt.RANDNX.run(n=np.full(10, 100).tolist(), input_shape=(1000, 1000), seed=42)

### RPROBNX

In [73]:
rprobnx = vbt.RPROBNX.run(entry_prob=1., exit_prob=1., input_shape=(5,), seed=42)

print(rprobnx.entries)
print(rprobnx.exits)

In [74]:
rprobnx = vbt.RPROBNX.run(
    entry_prob=np.asarray([1., 0., 1., 0., 1.]), 
    exit_prob=np.asarray([0., 1., 0., 1., 0.]), 
    input_shape=(5,), seed=42)

print(rprobnx.entries)
print(rprobnx.exits)

In [75]:
rprobnx = vbt.RPROBNX.run(entry_prob=[0.5, 1.], exit_prob=[1., 0.5], input_shape=(5,), seed=42)

print(rprobnx.entries)
print(rprobnx.exits)

In [76]:
%timeit vbt.RPROBNX.run(entry_prob=1., exit_prob=1., input_shape=(1000, 1000), seed=42)
%timeit vbt.RPROBNX.run(\
    entry_prob=np.full(10, 1.).tolist(), exit_prob=np.full(10, 1.).tolist(), \
    input_shape=(1000, 1000), seed=42)

### RPROBX

In [78]:
rprobx = vbt.RPROBX.run(entries, prob=[0., 0.5, 1.], seed=42)

print(rprobx.exits)

In [79]:
%timeit vbt.RPROBX.run(big_entries, prob=1., seed=42)
%timeit vbt.RPROBX.run(big_entries, prob=np.full(10, 1.).tolist(), seed=42)

### RPROBCX

In [80]:
rprobcx = vbt.RPROBCX.run(entries, prob=[0., 0.5, 1.], seed=42)

print(rprobcx.new_entries)
print(rprobcx.exits)

In [81]:
%timeit vbt.RPROBCX.run(big_entries, prob=1., seed=42)
%timeit vbt.RPROBCX.run(big_entries, prob=np.full(10, 1.).tolist(), seed=42)

### STX

In [82]:
stx = vbt.STX.run(entries, ts, 0.1)

print(stx.exits)

In [83]:
stx = vbt.STX.run(entries, ts, np.asarray([0.1, 0.1, -0.1, -0.1, -0.1])[:, None])

print(stx.exits)

In [84]:
stx = vbt.STX.run(entries, ts, [0.1, 0.1, -0.1, -0.1], trailing=[False, True, False, True])

print(stx.exits)

In [85]:
%timeit vbt.STX.run(big_entries, big_ts, 0.1)
%timeit vbt.STX.run(big_entries, big_ts, np.full(10, 0.1).tolist())

### STCX

In [86]:
stcx = vbt.STCX.run(entries, ts, [0.1, 0.1, -0.1, -0.1], trailing=[False, True, False, True])

print(stcx.new_entries)
print(stcx.exits)

In [87]:
%timeit vbt.STCX.run(big_entries, big_ts, 0.1)
%timeit vbt.STCX.run(big_entries, big_ts, np.full(10, 0.1).tolist())

### OHLCSTX

In [88]:
ohlcstx = vbt.OHLCSTX.run(
    entries, price['open'], price['high'], price['low'], price['close'], 
    sl_stop=0.1
)

print(ohlcstx.exits)
print(ohlcstx.stop_price)
print(ohlcstx.stop_type_readable)
ohlcstx[(0.1, 'b')].plot().show_svg()

In [89]:
ohlcstx = vbt.OHLCSTX.run(
    entries, price['open'], price['high'], price['low'], price['close'], 
    sl_stop=[0.1, 0., 0.], ts_stop=[0., 0.1, 0.], tp_stop=[0., 0., 0.1]
)

print(ohlcstx.exits)
print(ohlcstx.stop_price)
print(ohlcstx.stop_type_readable)

In [90]:
%timeit vbt.OHLCSTX.run(\
    big_entries, big_ts, big_ts + 1, big_ts - 1, big_ts,\
    sl_stop=0.1, ts_stop=0.1, tp_stop=0.1)
%timeit vbt.OHLCSTX.run(\
    big_entries, big_ts, big_ts + 1, big_ts - 1, big_ts,\
    sl_stop=np.full(10, 0.1).tolist(), ts_stop=np.full(10, 0.1).tolist(), tp_stop=np.full(10, 0.1).tolist())

### OHLCSTCX

In [91]:
ohlcstcx = vbt.OHLCSTCX.run(
    entries, price['open'], price['high'], price['low'], price['close'], 
    sl_stop=0.1
)

print(ohlcstcx.new_entries)
print(ohlcstcx.exits)
print(ohlcstcx.stop_price)
print(ohlcstcx.stop_type_readable)
ohlcstcx[(0.1, 'b')].plot().show_svg()

In [92]:
%timeit vbt.OHLCSTCX.run(\
    big_entries, big_ts, big_ts + 1, big_ts - 1, big_ts,\
    sl_stop=0.1, ts_stop=0.1, tp_stop=0.1)
%timeit vbt.OHLCSTCX.run(\
    big_entries, big_ts, big_ts + 1, big_ts - 1, big_ts,\
    sl_stop=np.full(10, 0.1).tolist(), ts_stop=np.full(10, 0.1).tolist(), tp_stop=np.full(10, 0.1).tolist())